<a href="https://colab.research.google.com/github/Manavjadiya/tweets-sentiment-/blob/main/Sentiment_Analysis_of_Tweeter_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer

In [17]:
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [18]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from sklearn import svm
from sklearn.metrics import classification_report
import csv
from sklearn.feature_extraction.text import TfidfVectorizer

tweetFile = pd.read_csv("tweets.csv")
dataFrame = pd.DataFrame(tweetFile[['content']])
tweetData = tweetFile['content']

In [19]:
tweetFile.head()

,author,content,country,date_time,id,language,latitude,longitude,number_of_likes,number_of_shares
0,katyperry,Is history repeating itself...?#DONTNORMALIZEH...,NaN,12/01/2017 19:52,8.196330e+17,en,NaN,NaN,7900,3472
1,katyperry,@barackobama Thank you for your incredible gra...,NaN,11/01/2017 08:38,8.191010e+17,en,NaN,NaN,3689,1380
2,katyperry,Life goals. https://t.co/XIn1qKMKQl,NaN,11/01/2017 02:52,8.190140e+17,en,NaN,NaN,10341,2387
3,katyperry,Me right now 🙏🏻 https://t.co/gW55C1wrwd,NaN,11/01/2017 02:44,8.190120e+17,en,NaN,NaN,10774,2458
4,katyperry,SISTERS ARE DOIN' IT FOR THEMSELVES! 🙌🏻💪🏻❤️ ht...,NaN,10/01/2017 05:22,8.186890e+17,en,NaN,NaN,17620,4655


In [ ]:
tknzr = TweetTokenizer()
stopWords = set(stopwords.words("english"))

In [ ]:
# words = word_tokenize(data[0]) #For 1 line

cleanedData = []
cleaned = []

for line in tweetData:
    tweet = tknzr.tokenize(str(line))

    for word in tweet:
        if word not in string.punctuation:
            if '@' not in word:
                cleaned.append(word)

    cleanedData.append(cleaned)
    cleaned = []

sentencedData = []

for sentence in cleanedData:
    sentencedData.append(" ".join(sentence))

tweetFile.insert(4, "clean_data", "")

cleanData = tweetFile['clean_data']
i = 0

for row in sentencedData:
    cleanData[i] = sentencedData[i]
    i = i + 1

loopData = [0, 1, 2, 3, 4]
time_linear_train = []
time_linear_predict = []

for loop in loopData:
    t0 = 0
    t1 = 0
    t2 = 0

    tweetDataCopy = tweetFile.copy()

    trainedTweetData = tweetDataCopy.sample(frac=.8, random_state=0)
    testTweetData = tweetDataCopy.drop(trainedTweetData.index)

    sid = SentimentIntensityAnalyzer()
    i = 0
    sentimentData = []

    for sentence in trainedTweetData['clean_data']:
        sentimentData.append(sid.polarity_scores(sentence)['compound'])

    sentimentLabel = []

    for sentiment in sentimentData:
        if sentiment >= 0.05:
            sentimentLabel.append("pos")
        elif sentiment <= -0.05:
            sentimentLabel.append("neg")
        else:
            sentimentLabel.append("neu")

    i = 0
    sentimentTestData = []

    for sentence in testTweetData['clean_data']:
        sentimentTestData.append(sid.polarity_scores(sentence)['compound'])

    sentimentForTestLabel = []

    for sentiment in sentimentTestData:
        if sentiment >= 0.05:
            sentimentForTestLabel.append("pos")
        elif sentiment <= -0.05:
            sentimentForTestLabel.append("neg")
        else:
            sentimentForTestLabel.append("neu")

In [ ]:
    data = {'clean_data': testTweetData.clean_data, 'sentiment': sentimentForTestLabel}
    df = pd.DataFrame(data)
    df.to_csv('test-data.csv')

    data = {'clean_data': trainedTweetData.clean_data, 'sentiment': sentimentLabel}
    df = pd.DataFrame(data)
    df.to_csv('train-data.csv')

    testData = pd.read_csv('test-data.csv')
    trainData = pd.read_csv('train-data.csv')

    # Create feature vectors
    vectorizer = TfidfVectorizer(min_df=5, max_df=0.8, sublinear_tf=True, use_idf=True)

    train_vectors = vectorizer.fit_transform(trainData['clean_data'].values.astype('U'))
    test_vectors = vectorizer.transform(testData['clean_data'].values.astype('U'))

    # Perform classification with SVM, kernel=linear
    classifier_linear = svm.SVC(kernel='linear')

    t0 = time.time()

    classifier_linear.fit(train_vectors, trainData['sentiment'])

    t1 = time.time()

    prediction_linear = classifier_linear.predict(test_vectors)

    t2 = time.time()

    time_linear_train.append(t1 - t0)
    time_linear_predict.append(t2 - t1)

    # results
    print("Training time: %fs; Prediction time: %fs" % (time_linear_train[loop], time_linear_predict[loop]))
    report = classification_report(testData['sentiment'], prediction_linear, output_dict=True)

    print('positive: ', report['pos'])
    print('negative: ', report['neg'])

totalTrainTime = 0
totalPredictTime = 0

#for i in loopData:
     # for j in loopData:
       #  if time_linear_train[i] < time_linear_train[j]:
        #   minTimeOfLinearTrain = time_linear_train[i]
        # else:
       #    minTimeOfLinearTrain = time_linear_train[j]

       #  if time_linear_predict[i] < time_linear_predict[j]:
       #    minTimeOfLinearPredict = time_linear_predict[i]
       #  else:
       #    minTimeOfLinearPredict = time_linear_predict[j]
  # totalTrainTime = totalTrainTime + time_linear_train[i]
  # totalPredictTime = totalPredictTime + time_linear_predict[i]

print("Average training time: %fs" % (totalTrainTime / 5))
print("Average prediction time: %fs" % (totalPredictTime / 5))